In [1]:
import xml.etree.ElementTree as etree
import codecs
import csv
import time
import os
import re

In [2]:
PATH_WIKI_XML = os.getcwd()
FILENAME_WIKI = 'enwiki-latest-pages-articles.xml'
FILENAME_ARTICLES = 'articles.csv'
FILENAME_REDIRECT = 'articles_redirect.csv'
FILENAME_TEMPLATE = 'articles_template.csv'
FILENAME_DRUG = 'articles_drugs.csv'
FILENAME_DISEASE = 'articles_diseases.csv'
ENCODING = "utf-8"

In [3]:
# Nicely formatted time string
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)
def strip_tag_name(t):
    t = elem.tag
    idx = k = t.rfind("}")
    if idx != -1:
        t = t[idx + 1:]
    return t
def get_drugbox(s):
    beg = (s.rfind('{{Drugbox'))
    end  =(s.rfind('\n}}'))
    if( end == -1):
        end = end =(s.rfind('}}\n'))
    s = s[beg: end+2]
    return s
def get_medical(s):
    beg = (s.rfind('{{Medical resources'))
    end  =(s.rfind('\n}}'))
    s = s[beg: end+3]
    return s
def find_cas(s):
    s = re.findall(r'CAS_number = [0-9]*-?[0-9]*-?[0-9]*',s)
    if(len(s)>0):   
        s = s[0]
        s = s[13:]
    else:
        s = 'NOT FOUND'
    return s

In [4]:
pathWikiXML =FILENAME_WIKI
pathArticles = FILENAME_ARTICLES
pathArticlesRedirect =  FILENAME_REDIRECT
pathTemplateRedirect =  FILENAME_TEMPLATE
pathDrugsArticles =FILENAME_DRUG
pathDiseaseArticles = FILENAME_DISEASE

In [5]:
totalCount = 0
articleCount = 0
redirectCount = 0
templateCount = 0
title = None
start_time = time.time()

with codecs.open(pathDrugsArticles, "w", ENCODING) as articlesFH:
    articlesWriter = csv.writer(articlesFH, quoting=csv.QUOTE_MINIMAL)

    articlesWriter.writerow(['id', 'title'])
    for event, elem in etree.iterparse(pathWikiXML, events=('start', 'end')):
        tname = strip_tag_name(elem.tag)

        if event == 'start':
            if tname == 'page':
                title = ''
                id = -1
                redirect = ''
                inrevision = False
                ns = 0
            elif tname == 'revision':
                # Do not pick up on revision id's
                inrevision = True
        else:
            if tname == 'title':
                title = elem.text
            elif tname == 'id' and not inrevision:
                id = int(elem.text)
            elif tname == 'redirect':
                redirect = elem.attrib['title']
            elif tname == 'ns':
                ns = int(elem.text)
            elif tname == 'page':
                totalCount += 1
            elif(tname =='text'):
                cont = elem.text
                #check if cont exists
                if(type(cont)==type('alex')):
                    #check if the drugbox template exists 
                    if (cont.find('Drugbox')>-1):
                        #get the drugbox template
                        result = get_drugbox(cont)
                        #get the CAS number
                        cas = find_cas(result)
                        if(cas == 'NOT FOUND'):
                            print(title)
                        articlesWriter.writerow([id, title, cas])
                        print('done')
            
    elem.clear()
elapsed_time = time.time() - start_time

print("Total pages: {:,}".format(totalCount))
print("Template pages: {:,}".format(templateCount))
print("Article pages: {:,}".format(articleCount))
print("Redirect pages: {:,}".format(redirectCount))
print("Elapsed time: {}".format(hms_string(elapsed_time)))

done
done
done
done
done
done
done
done
done
done


KeyboardInterrupt: 

In [7]:
totalCount = 0
articleCount = 0
redirectCount = 0
templateCount = 0
title = None
start_time = time.time()

with codecs.open(pathDiseaseArticles, "w", ENCODING) as articlesFH:
    articlesWriter = csv.writer(articlesFH, quoting=csv.QUOTE_MINIMAL)

    articlesWriter.writerow(['id', 'title'])
    for event, elem in etree.iterparse(pathWikiXML, events=('start', 'end')):
        tname = strip_tag_name(elem.tag)

        if event == 'start':
            if tname == 'page':
                title = ''
                id = -1
                redirect = ''
                inrevision = False
                ns = 0
            elif tname == 'revision':
                # Do not pick up on revision id's
                inrevision = True
        else:
            if tname == 'title':
                title = elem.text
            elif tname == 'id' and not inrevision:
                id = int(elem.text)
            elif tname == 'redirect':
                redirect = elem.attrib['title']
            elif tname == 'ns':
                ns = int(elem.text)
            elif tname == 'page':
                totalCount += 1
            elif(tname =='text'):
                cont = elem.text
                #check if cont exists
                if(type(cont)==type('alex')):
                    #check if the drugbox template exists 
                    if (cont.find('Medical resources')>-1):
                        #get the drugbox template
                        result = get_medical(cont)
                        #get the CAS number
                        id10 = re.findall('{{ICD9.*',result)
                        print(id10)
                        articlesWriter.writerow([id, title, id10])
                        print('done')
            
    elem.clear()
elapsed_time = time.time() - start_time

print("Total pages: {:,}".format(totalCount))
print("Template pages: {:,}".format(templateCount))
print("Article pages: {:,}".format(articleCount))
print("Redirect pages: {:,}".format(redirectCount))
print("Elapsed time: {}".format(hms_string(elapsed_time)))

['{{ICD9|299.00}}']
done
['{{ICD9|335.2}} ']
done
['{{ICD9|323.61}}, {{ICD9|323.81}}']
done
['{{ICD9|xxx}}-->']
done
['{{ICD9|710}}-{{ICD9|719}}']
done
['{{ICD9|270.2}}']
done
['{{ICD9|747.6}}, {{ICD9|747.81}} ']
done
['{{ICD9|324.0}} ']
done
['{{ICD9|616.1}} ']
done
['{{ICD9|020}}']
done
['{{ICD9|296.7}} ']
done
['{{ICD9|046.1}}']
done
['{{ICD9|780.01}}']
done
['{{ICD9|427.81}}, {{ICD9|659.7}}, {{ICD9|785.9}}, {{ICD9|779.81}}']
done
['{{ICD9|243}} ']
done
['{{ICD9|112}}']
done
['{{ICD9|747.81}}']
done
['{{ICD9|356.1}} ']
done
['{{ICD9|xxx}}-->']
done
['{{ICD9|305.0}}, {{ICD9|980}}']
done
['{{ICD9|421}}']
done
['{{ICD9|315.31}}']
done
['{{ICD9|345}}']
done
['{{ICD9|333.1}}']
done
['{{ICD9|300.13}} ']
done
['{{ICD9|336.1}} ']
done
['{{ICD9|240.9}} ']
done
['{{ICD9|342.9}}, {{ICD9|094.89}}, {{ICD9|438.2}} ']
done
['{{ICD9|302.72}}, {{ICD9|607.84}}']
done
['{{ICD9|595.1}}']
done
['{{ICD9|359.71}} ']
done
['{{ICD9|260}}']
done
['{{ICD9|xxx}}-->']
done
['{{ICD9|333.2}} ']
done
['{{ICD9|162}

KeyboardInterrupt: 

In [ ]:
s= conts[1]
beg = (s.rfind('{{ICD10'))
end = s.rfind('\n|.* ICD9')
print(beg, end)
s = s[beg:end-1]

In [ ]:
s= conts[3]
alex = re.findall('{{ICD10.*',s)
print(alex)

In [ ]:
for i in range(0,10):
    s= conts[i]
    alex = re.findall('{{ICD9.*',s)
    print(alex, titles[i])
    